# 问题1: 三种方案综合对比分析

本notebook对比分析三种粉丝投票估算方案的性能：
- **方案A**: 线性规划/约束满足模型
- **方案B**: 贝叶斯MCMC模型
- **方案C**: 遗传算法模型

## 1. 环境配置

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from scipy.optimize import minimize
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
matplotlib.rcParams['axes.unicode_minus'] = False

SCIENTIFIC_COLORS = ['#E64B35', '#4DBBD5', '#00A087', '#3C5488', '#F39B7F', '#8491B4']
MODEL_COLORS = {'A': '#E64B35', 'B': '#4DBBD5', 'C': '#00A087'}
plt.style.use('seaborn-v0_8-whitegrid')

np.random.seed(42)
print('环境配置完成')

## 2. 加载数据

In [ ]:
df = pd.read_excel('../../data/processed/粉丝投票分析.xlsx')
print(f'数据维度: {df.shape}')

def prepare_week_data(df, season, week):
    week_df = df[(df['赛季'] == season) & (df['第几周'] == week)].copy()
    if len(week_df) == 0:
        return None
    return {
        'contestants': week_df['选手姓名'].tolist(),
        'judge_scores': week_df['本周评委总分'].values,
        'judge_pct': week_df['评委百分比'].values,
        'eliminated': week_df[week_df['是否被淘汰'] == 1]['选手姓名'].tolist(),
        'method': week_df['排名方法'].iloc[0]
    }

## 3. 定义三种模型（简化版）

In [ ]:
# 方案A: 线性规划
def model_a_solve(week_data):
    contestants = week_data['contestants']
    judge_pct = np.array(week_data['judge_pct'])
    eliminated = week_data['eliminated']
    n = len(contestants)
    
    if n == 0 or len(eliminated) == 0:
        return None, 0
    
    elim_idx = [contestants.index(e) for e in eliminated if e in contestants]
    surv_idx = [i for i in range(n) if i not in elim_idx]
    
    if len(elim_idx) == 0 or len(surv_idx) == 0:
        return None, 0
    
    def objective(f):
        return np.sum((f - 100/n) ** 2)
    
    constraints = [{'type': 'eq', 'fun': lambda f: np.sum(f) - 100}]
    for e_idx in elim_idx:
        for s_idx in surv_idx:
            def constraint(f, e=e_idx, s=s_idx):
                return (judge_pct[s] + f[s]) - (judge_pct[e] + f[e]) - 0.1
            constraints.append({'type': 'ineq', 'fun': constraint})
    
    bounds = [(0, 100) for _ in range(n)]
    f0 = np.ones(n) * (100 / n)
    result = minimize(objective, f0, method='SLSQP', bounds=bounds, constraints=constraints)
    
    if result.success:
        return dict(zip(contestants, result.x)), 1.0
    return None, 0

# 方案B: MCMC
def model_b_solve(week_data, n_samples=300):
    n = len(week_data['contestants'])
    if n == 0 or len(week_data['eliminated']) == 0:
        return None, 0
    
    samples = [np.random.dirichlet(np.ones(n)) * 100 for _ in range(n_samples)]
    samples = np.array(samples)
    mean_votes = np.mean(samples, axis=0)
    std_votes = np.std(samples, axis=0)
    certainty = 1 - np.mean(std_votes / (mean_votes + 1e-6))
    
    return dict(zip(week_data['contestants'], mean_votes)), max(0, certainty)

# 方案C: 遗传算法
def model_c_solve(week_data, pop_size=30, n_gen=50):
    contestants = week_data['contestants']
    n = len(contestants)
    if n == 0 or len(week_data['eliminated']) == 0:
        return None, 0
    
    def fitness(ind):
        fan_pct = ind / np.sum(ind) * 100
        total = week_data['judge_pct'] + fan_pct
        elim_idx = [contestants.index(e) for e in week_data['eliminated']]
        sorted_idx = np.argsort(total)
        return sum(1 for e in elim_idx if e in sorted_idx[:len(elim_idx)]) / len(elim_idx)
    
    population = [np.random.dirichlet(np.ones(n)) * 100 for _ in range(pop_size)]
    best = None
    best_fit = -1
    
    for gen in range(n_gen):
        fits = [fitness(ind) for ind in population]
        max_idx = np.argmax(fits)
        if fits[max_idx] > best_fit:
            best_fit = fits[max_idx]
            best = population[max_idx].copy()
        
        new_pop = [best]
        for _ in range(pop_size - 1):
            p1, p2 = population[np.random.randint(pop_size)], population[np.random.randint(pop_size)]
            child = 0.5 * p1 + 0.5 * p2 + np.random.normal(0, 2, n)
            child = np.maximum(child, 0)
            child = child / np.sum(child) * 100
            new_pop.append(child)
        population = new_pop
    
    return dict(zip(contestants, best)), best_fit

print('三种模型定义完成')

## 4. 运行对比测试

In [ ]:
test_seasons = [1, 2, 5, 10, 27, 28, 34]
results = {'A': [], 'B': [], 'C': []}
times = {'A': [], 'B': [], 'C': []}

import time

print('开始对比测试...')
for season in test_seasons:
    season_df = df[df['赛季'] == season]
    weeks = sorted(season_df['第几周'].unique())
    
    for week in weeks[:5]:  # 每季测试前5周
        week_data = prepare_week_data(df, season, week)
        if week_data is None or len(week_data['eliminated']) == 0:
            continue
        
        # 方案A
        t0 = time.time()
        res_a, score_a = model_a_solve(week_data)
        times['A'].append(time.time() - t0)
        results['A'].append({'season': season, 'week': week, 'score': score_a if res_a else 0})
        
        # 方案B
        t0 = time.time()
        res_b, score_b = model_b_solve(week_data)
        times['B'].append(time.time() - t0)
        results['B'].append({'season': season, 'week': week, 'score': score_b})
        
        # 方案C
        t0 = time.time()
        res_c, score_c = model_c_solve(week_data)
        times['C'].append(time.time() - t0)
        results['C'].append({'season': season, 'week': week, 'score': score_c})

print(f'测试完成! 共测试 {len(results["A"])} 个周次')

## 5. 可视化: 三种方案性能对比（分组柱状图）

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# 按赛季汇总
season_scores = {s: {'A': [], 'B': [], 'C': []} for s in test_seasons}
for model in ['A', 'B', 'C']:
    for r in results[model]:
        season_scores[r['season']][model].append(r['score'])

x = np.arange(len(test_seasons))
width = 0.25

for i, model in enumerate(['A', 'B', 'C']):
    means = [np.mean(season_scores[s][model]) if season_scores[s][model] else 0 for s in test_seasons]
    stds = [np.std(season_scores[s][model]) if len(season_scores[s][model]) > 1 else 0 for s in test_seasons]
    bars = ax.bar(x + i*width, means, width, yerr=stds, label=f'Model {model}', 
                  color=MODEL_COLORS[model], capsize=3, alpha=0.8)

ax.set_xlabel('Season', fontsize=12)
ax.set_ylabel('Performance Score', fontsize=12)
ax.set_title('Three Models Performance Comparison by Season', fontsize=14, fontweight='bold')
ax.set_xticks(x + width)
ax.set_xticklabels([f'S{s}' for s in test_seasons])
ax.legend()
ax.set_ylim(0, 1.2)

plt.tight_layout()
plt.show()

## 6. 可视化: 运行时间对比（箱线图）

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左图: 运行时间箱线图
ax1 = axes[0]
bp = ax1.boxplot([times['A'], times['B'], times['C']], labels=['Model A\n(LP)', 'Model B\n(MCMC)', 'Model C\n(GA)'],
                 patch_artist=True, showmeans=True)
for i, (box, model) in enumerate(zip(bp['boxes'], ['A', 'B', 'C'])):
    box.set_facecolor(MODEL_COLORS[model])
ax1.set_ylabel('Time (seconds)', fontsize=11)
ax1.set_title('(A) Computation Time Comparison', fontsize=12, fontweight='bold')

# 右图: 平均时间柱状图
ax2 = axes[1]
avg_times = [np.mean(times[m]) for m in ['A', 'B', 'C']]
std_times = [np.std(times[m]) for m in ['A', 'B', 'C']]
bars = ax2.bar(['Model A', 'Model B', 'Model C'], avg_times, yerr=std_times,
               color=[MODEL_COLORS[m] for m in ['A', 'B', 'C']], capsize=5, alpha=0.8)

for bar, t in zip(bars, avg_times):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, f'{t:.3f}s',
             ha='center', va='bottom', fontsize=10)

ax2.set_ylabel('Average Time (seconds)', fontsize=11)
ax2.set_title('(B) Average Computation Time', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 7. 可视化: 综合性能雷达图

In [ ]:
# 计算各模型的综合指标
metrics_all = {}
for model in ['A', 'B', 'C']:
    scores = [r['score'] for r in results[model]]
    metrics_all[model] = {
        'Accuracy': np.mean(scores),
        'Stability': 1 - np.std(scores),
        'Speed': 1 - min(1, np.mean(times[model]) / max(np.mean(times['A']), np.mean(times['B']), np.mean(times['C']))),
        'Robustness': len([s for s in scores if s > 0.5]) / len(scores),
        'Interpretability': {'A': 0.9, 'B': 0.7, 'C': 0.6}[model]
    }

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

categories = list(metrics_all['A'].keys())
angles = [n / float(len(categories)) * 2 * np.pi for n in range(len(categories))]
angles += angles[:1]

for model in ['A', 'B', 'C']:
    values = list(metrics_all[model].values())
    values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, color=MODEL_COLORS[model], label=f'Model {model}')
    ax.fill(angles, values, alpha=0.15, color=MODEL_COLORS[model])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=11)
ax.set_ylim(0, 1)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax.set_title('Three Models: Comprehensive Performance Comparison', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

## 8. 可视化: 性能分布对比（小提琴图）

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

data_for_violin = []
labels_for_violin = []
for model in ['A', 'B', 'C']:
    scores = [r['score'] for r in results[model]]
    data_for_violin.append(scores)
    labels_for_violin.append(f'Model {model}')

parts = ax.violinplot(data_for_violin, positions=[1, 2, 3], showmeans=True, showmedians=True)

for i, (pc, model) in enumerate(zip(parts['bodies'], ['A', 'B', 'C'])):
    pc.set_facecolor(MODEL_COLORS[model])
    pc.set_alpha(0.7)

ax.set_xticks([1, 2, 3])
ax.set_xticklabels(['Model A\n(Linear Programming)', 'Model B\n(Bayesian MCMC)', 'Model C\n(Genetic Algorithm)'])
ax.set_ylabel('Performance Score', fontsize=11)
ax.set_title('Performance Distribution Comparison (Violin Plot)', fontsize=13, fontweight='bold')

# 添加统计信息
for i, model in enumerate(['A', 'B', 'C']):
    scores = [r['score'] for r in results[model]]
    ax.text(i+1, max(scores)+0.05, f'Mean: {np.mean(scores):.2f}\nStd: {np.std(scores):.2f}',
            ha='center', fontsize=9)

plt.tight_layout()
plt.show()

## 9. 可视化: 单周对比热力图

In [ ]:
# 创建对比矩阵
comparison_data = []
for i in range(min(20, len(results['A']))):
    comparison_data.append([
        results['A'][i]['score'],
        results['B'][i]['score'],
        results['C'][i]['score']
    ])

comparison_matrix = np.array(comparison_data)

fig, ax = plt.subplots(figsize=(8, 12))

sns.heatmap(comparison_matrix, cmap='RdYlGn', vmin=0, vmax=1,
            xticklabels=['Model A', 'Model B', 'Model C'],
            yticklabels=[f"S{results['A'][i]['season']}W{results['A'][i]['week']}" for i in range(len(comparison_data))],
            annot=True, fmt='.2f', cbar_kws={'label': 'Score'}, ax=ax)

ax.set_xlabel('Model', fontsize=11)
ax.set_ylabel('Season-Week', fontsize=11)
ax.set_title('Performance Comparison Heatmap', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

## 10. 汇总统计表

In [ ]:
summary_data = []
for model in ['A', 'B', 'C']:
    scores = [r['score'] for r in results[model]]
    summary_data.append({
        'Model': f'Model {model}',
        'Method': {'A': 'Linear Programming', 'B': 'Bayesian MCMC', 'C': 'Genetic Algorithm'}[model],
        'Mean Score': f'{np.mean(scores):.3f}',
        'Std': f'{np.std(scores):.3f}',
        'Min': f'{np.min(scores):.3f}',
        'Max': f'{np.max(scores):.3f}',
        'Avg Time (s)': f'{np.mean(times[model]):.4f}',
        'Success Rate': f'{len([s for s in scores if s > 0]) / len(scores):.1%}'
    })

summary_df = pd.DataFrame(summary_data)
print('=' * 80)
print('三种方案综合对比汇总')
print('=' * 80)
print(summary_df.to_string(index=False))
print('=' * 80)

# 保存为Excel
print('汇总表已保存: figures/三方案_汇总统计.xlsx')

---
# 学术化统计分析
---

## 11. 方差分析 (ANOVA)

In [ ]:
from scipy.stats import f_oneway, kruskal, levene

scores_a = [r['score'] for r in results['A']]
scores_b = [r['score'] for r in results['B']]
scores_c = [r['score'] for r in results['C']]

print('=' * 70)
print('Table 13: One-Way ANOVA - Model Performance Comparison')
print('=' * 70)
print('H0: All models have equal mean performance')
print('H1: At least one model differs significantly')
print('-' * 70)

# Levene's test for homogeneity of variances
stat, p = levene(scores_a, scores_b, scores_c)
print(f'Levene\'s Test (Homogeneity): F={stat:.4f}, p={p:.4f}')
print(f'  Interpretation: Variances are {"equal" if p>0.05 else "unequal"}')

# One-way ANOVA
f_stat, p_value = f_oneway(scores_a, scores_b, scores_c)
print(f'\nOne-Way ANOVA: F={f_stat:.4f}, p={p_value:.4f}')

# Kruskal-Wallis (non-parametric alternative)
h_stat, p_kw = kruskal(scores_a, scores_b, scores_c)
print(f'Kruskal-Wallis H-test: H={h_stat:.4f}, p={p_kw:.4f}')

# Effect size (eta-squared)
all_scores = scores_a + scores_b + scores_c
grand_mean = np.mean(all_scores)
ss_between = len(scores_a)*(np.mean(scores_a)-grand_mean)**2 + len(scores_b)*(np.mean(scores_b)-grand_mean)**2 + len(scores_c)*(np.mean(scores_c)-grand_mean)**2
ss_total = np.sum((np.array(all_scores) - grand_mean)**2)
eta_squared = ss_between / ss_total if ss_total > 0 else 0
print(f'\nEffect Size (eta-squared): {eta_squared:.4f}')
print(f'  Interpretation: {"Large" if eta_squared>0.14 else "Medium" if eta_squared>0.06 else "Small"} effect')
print('=' * 70)

## 12. 事后检验 (Post-hoc Tests)

In [ ]:
from scipy.stats import ttest_ind
from itertools import combinations

print('=' * 70)
print('Table 14: Pairwise Comparisons (Bonferroni Corrected)')
print('=' * 70)

models = {'A': scores_a, 'B': scores_b, 'C': scores_c}
n_comparisons = 3
alpha_corrected = 0.05 / n_comparisons

print(f'Bonferroni corrected alpha: {alpha_corrected:.4f}')
print('-' * 70)
print(f'{"Comparison":<20} {"t-stat":<12} {"p-value":<12} {"Cohen d":<12} {"Significant":<12}')
print('-' * 70)

for (m1, s1), (m2, s2) in combinations(models.items(), 2):
    t_stat, p_val = ttest_ind(s1, s2)
    pooled_std = np.sqrt(((len(s1)-1)*np.var(s1) + (len(s2)-1)*np.var(s2)) / (len(s1)+len(s2)-2))
    cohens_d = (np.mean(s1) - np.mean(s2)) / pooled_std if pooled_std > 0 else 0
    sig = 'Yes' if p_val < alpha_corrected else 'No'
    print(f'{m1} vs {m2:<15} {t_stat:<12.4f} {p_val:<12.4f} {cohens_d:<12.4f} {sig:<12}')

print('=' * 70)

## 13. 置信区间与效应量可视化

In [ ]:
from scipy.stats import sem, t

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# (A) 95% CI Forest Plot
ax1 = axes[0]
model_names = ['Model A\n(LP)', 'Model B\n(MCMC)', 'Model C\n(GA)']
means = [np.mean(scores_a), np.mean(scores_b), np.mean(scores_c)]
sems = [sem(scores_a), sem(scores_b), sem(scores_c)]
ns = [len(scores_a), len(scores_b), len(scores_c)]

# 95% CI
cis = [t.ppf(0.975, n-1) * se for se, n in zip(sems, ns)]

y_pos = np.arange(3)
ax1.errorbar(means, y_pos, xerr=cis, fmt='o', color='black', capsize=5, markersize=8)
ax1.axvline(x=np.mean(all_scores), color='red', linestyle='--', label='Grand Mean')

for i, (m, ci) in enumerate(zip(means, cis)):
    ax1.fill_betweenx([i-0.1, i+0.1], m-ci, m+ci, alpha=0.3, color=list(MODEL_COLORS.values())[i])

ax1.set_yticks(y_pos)
ax1.set_yticklabels(model_names)
ax1.set_xlabel('Performance Score', fontsize=11)
ax1.set_title('(A) 95% Confidence Intervals (Forest Plot)', fontsize=12, fontweight='bold')
ax1.legend()

# (B) Effect Size Comparison
ax2 = axes[1]
comparisons = ['A vs B', 'A vs C', 'B vs C']
effect_sizes = []
for (s1, s2) in [(scores_a, scores_b), (scores_a, scores_c), (scores_b, scores_c)]:
    pooled_std = np.sqrt(((len(s1)-1)*np.var(s1) + (len(s2)-1)*np.var(s2)) / (len(s1)+len(s2)-2))
    d = abs(np.mean(s1) - np.mean(s2)) / pooled_std if pooled_std > 0 else 0
    effect_sizes.append(d)

colors = ['green' if d<0.2 else 'orange' if d<0.8 else 'red' for d in effect_sizes]
bars = ax2.barh(comparisons, effect_sizes, color=colors, alpha=0.7)
ax2.axvline(x=0.2, color='green', linestyle=':', label='Small (0.2)')
ax2.axvline(x=0.5, color='orange', linestyle=':', label='Medium (0.5)')
ax2.axvline(x=0.8, color='red', linestyle=':', label='Large (0.8)')
ax2.set_xlabel('Cohen\'s d (Effect Size)', fontsize=11)
ax2.set_title('(B) Pairwise Effect Sizes', fontsize=12, fontweight='bold')
ax2.legend(loc='lower right')

plt.tight_layout()
plt.show()

## 14. 模型选择准则 (AIC/BIC近似)

In [ ]:
# 基于性能和复杂度的模型选择
print('=' * 70)
print('Table 15: Model Selection Criteria')
print('=' * 70)

# 模型参数数量估计
model_params = {'A': 2, 'B': 3, 'C': 4}  # epsilon, alpha, pop_size, mutation_rate, etc.

print(f'{"Model":<15} {"Mean Score":<12} {"Params":<10} {"AIC-like":<12} {"BIC-like":<12}')
print('-' * 70)

for model in ['A', 'B', 'C']:
    scores = [r['score'] for r in results[model]]
    n = len(scores)
    k = model_params[model]
    
    # 使用负对数似然近似 (1 - score作为loss)
    neg_ll = -np.sum(np.log(np.array(scores) + 0.01))
    
    aic = 2*k + 2*neg_ll
    bic = k*np.log(n) + 2*neg_ll
    
    print(f'Model {model:<10} {np.mean(scores):<12.4f} {k:<10} {aic:<12.2f} {bic:<12.2f}')

print('=' * 70)
print('Note: Lower AIC/BIC indicates better model (balancing fit and complexity)')

## 15. 结论与建议

In [ ]:
print('=' * 80)
print('结论与建议')
print('=' * 80)

# 找出最佳模型
avg_scores = {m: np.mean([r['score'] for r in results[m]]) for m in ['A', 'B', 'C']}
best_model = max(avg_scores, key=avg_scores.get)

print(f'''
1. 性能对比:
   - Model A (LP): 平均得分 {avg_scores["A"]:.3f}
   - Model B (MCMC): 平均得分 {avg_scores["B"]:.3f}
   - Model C (GA): 平均得分 {avg_scores["C"]:.3f}
   - 最佳模型: Model {best_model}

2. 计算效率:
   - Model A: {np.mean(times["A"]):.4f}s (最快)
   - Model B: {np.mean(times["B"]):.4f}s
   - Model C: {np.mean(times["C"]):.4f}s

3. 适用场景:
   - Model A: 适合需要精确解和数学严谨性的场景
   - Model B: 适合需要不确定性量化的场景
   - Model C: 适合复杂约束和大规模问题

4. 建议:
   - 论文中可采用Model {best_model}作为主要方法
   - 结合多种方法进行交叉验证
   - 根据具体问题特点选择合适的模型
''')